In [6]:
import gurobipy as gp
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math

## Generazione dataset

Prima genera tutti i CC e gli SP che non appartengono a cluster, sono distribuiti uniformemente nella mappa.
Poi seleziona 10 CC che saranno il centro dei cluster, per ognuno di essi genera le coordinate degli SP (uniformemente), nella regione inclusa fra +- 3 punti di distanza. Tutti i cluster hanno stesso numero di SP

In [7]:
#la funzione prende in input: k dimensioni mappa, N_SP numero di source points, n_cluster numero di cluster 
#e cluster ratio percentuale di punti facenti parte di un cluster
#ritorna in output vettori numpy contenenti le coordinate dei punti
def data_generation(k, n_SP, n_CC, n_cluster, cluster_ratio):
    
    #definisco coordinate di tutti i CC e degli SP non facenti parte dei cluster
    P=np.random.randint(0, k+1, size=(n_CC, 2))  #genera il set dei CC
    S_no_cluster=np.random.randint(0, k+1, size=(n_SP-int(cluster_ratio*n_SP), 2))  #genera il set dei SP non facenti parte di un cluster

    #genero coordinate SP rimanenti
    cluster_index=random.sample(range(0,n_CC), int(n_CC*cluster_ratio)) #seleziona l'indice dei CC a caso che diventeranno i centri dei cluster
    S_cluster_list=[]  #creo lista vuota da popolare con i numpy array e poi concatenarli TROVA MODO PIU EFFICIENTE

    #per ognuno dei cluster genero casualmente uniformemente le coordinate (entro i 36 punti adiacenti) di 5 SP
    for i in cluster_index:
        #voglio considerare il contorno dei 36 vicini, quindi indici (i-3, j-3) (i+3, j+3)
        xmin=P[i][0]-3
        xmax=P[i][0]+3

        ymin=P[i][1]-3
        ymax=P[i][1]+3
    
        #considera ogni cluster con stesso numero di SP
        S_cluster_list.append(np.random.randint(low=[xmin, ymin], high=[xmax, ymax], size=(round(cluster_ratio*n_SP/n_cluster), 2)))

    #mette assieme il tutto
    S_cluster=np.concatenate(S_cluster_list) #crea array unico per i cluster
    S=np.concatenate((S_cluster, S_no_cluster))

    #genera posizione della fabbrica biodiesel
    biodiesel=np.random.randint(0, k+1, size=(1, 2))
    
    return P, S, biodiesel

In [8]:
#funzione che date le coordinate di due punti (in array 2d numpy) ritorna la distanza euclidea arrotondata in integer
def distance(a, b):
    dist=round(math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2))
    return dist

## Constructive heuristic

Fase 1 focus su CC location, SP assignment e bin allocation.
Fase 2 si occupa del percorso dei veicoli.

in 1 definisco attractiveness score per trovare set CC candidati per poi assegnare gli SP e i bin ai CC selezionati.
in 2 si genera il percorso utilizzando algoritmo modificato di Clarke-Wright che tiene conto anche delle maximum route constraints.

In [9]:
k=1000  #dimensioni della mappa (prima test con k=1000, poi varialo per studio domansionale)

n_SP=100   #number of source points
n_CC=20 #number of collection centers
n_cluster=int(0.5*n_CC)  #number of clusters
cluster_ratio=0.5  #how many of the total nodes (SPs and CCs) fall within a cluster

#RICORDATI DI COMMENTARE IL SEED PER MISURARE PERFORMANCE
np.random.seed(seed=420) #fissa il seed per riproducibilità nei test

P, S, biodiesel=data_generation(k, n_SP, n_CC, n_cluster, cluster_ratio)


#definizione e generazione delle altre costanti
alpha=1  #costo per unità di distanza
V=5  #numero di veicoli MESSO A CASO, NEL PAPER NON TROVO QUANTO DOVREBBE ESSERE
Q=24  #numero di cestini su ogni veicolo
B=50  #capacità, in litri, dei bins sui veicoli
b=10  #costo di un bin
max_distance=300  #distanza massima fra un SP e un CC
R=1800 #lungezza massima percorso per veicolo, se distanza fra 0,0 e 1000,1000 è 1414 allora 1800 non è sufficiente nel caso il centrp biodiesel venga generato in un angolo?
P0=np.concatenate((P, biodiesel)) #set posizione biodiesel+CC (serve in calcoli veicoli)
f=np.random.randint(50, 150+1, size=(n_CC))  #costo per aprire il CC i
d_s=np.random.randint(1, 20+1, size=(len(S)))  #produzione settimanale in litri di olio del SP i
C=np.random.randint(1, 5+1, size=(n_CC))  #numero di Bins nel CC i


#P_s è il set dei CC entro la distanza di copertura dal SP s
#quindi per ogni SP creo lista P_s contente indice dei CC compatibili, poi salvo in lista di liste il tutto
P_s_list=[]  #inizializzo lista che conterrà i vari P_s
for s in range(len(S)):
    P_s=[]  #inizializzo P_s
    for i in range(len(P)): 
        if (distance(S[s], P[i])<300):   #controlla se la distanza è minore di 300
            P_s.append(i)  #se distanza e minore di 300 salva l'indice del CC compatibile
    P_s_list.append(P_s)  #aggiunge la lista alla lista totale


#crea matrice delle distanze fra elementi di P0
c0=np.empty([len(P0), len(P0)])
for i in range(len(P0)):
    for j in range(len(P0)):
        c0[i,j]=distance(P0[i], P0[j])

#crea matrice delle distanze fra elementi di S e P
cs=np.empty([len(S), len(P)])
for i in range(len(S)):
    for j in range(len(P)):
        cs[i,j]=distance(S[i], P[j])        

In [10]:
#crea i set degli indici
P_set=range(0, n_CC) #CC
S_set=range(0, n_SP) #SP
P0_set=range(0, n_CC+1)  #CC+centro biodiesel (ultimo elemento)
V_set=range(0, V)  #veicoli


beta=[] #salva le soluzioni appena trovate in lista. 
#algoritmo 1
for omega in np.linspace(0,1,10):
    for n in range(1, math.ceil(n_CC/2)): #nel paper n parte da 0, ma in calcolo attractiveness è presente una divisione per n

        P_hat=[] #set dei CC candidati
        S_to_exc=[] #indice degli SP con soluzione unica
        #prima identifico gli SP che possono essere connessi a un solo CC, che dovrà per forza essere aperto e quindi aggiunto a P_hat
        for i in range(len(P_s_list)):
            if (len(P_s_list[i])==1 and P_s_list[i][0] not in P_hat): #controlla se lista SP ha un solo CC vicino e se il CC non è già stato aggiunto a P_hat
                P_hat.append(P_s_list[i][0])
                S_to_exc.append(i)
        
        S_ch_set=set(S_set)-set(S_to_exc) #eslude SP con un unico CC vicino
        CC_ch_set=set(P_set)-set(P_hat) #esclude i CC assegnati all'SP di cui sopra

        #calcolo attractiveness
        A=np.empty(shape=(n_CC))
        #omega, n sono parametri da settare successivamente; intanto sono fissati per test

        #set dei closest CC to CC_i
        CC_closest_list=[]  #inizializzo lista che conterrà i vari set dei closest CC
        for index in range(n_CC):
            CC_closest_list.append([i[0] for i in sorted(enumerate(c0[index,:]), key=lambda x:x[1])]) #ritorna lista indici ordinati per distanza 
            #il primo elemento di ogni lista di CC_closest_list è il CC stesso, quindi va ignorato    

        #per costruire P_hat:
        while len(S_ch_set)!=0:  #finchè non vengono assegnati tutti gli SP
            for i in range(n_CC): #calcola attractiveness
                if i in P_hat:
                    A[i]=0  #per evitare di selezionare CC già in P_hat
                else:    
                    A[i]=min(C[i], sum(d_s[s] for s in S_set if i in P_s_list[s]))/(omega*c0[i,len(P0)-1]+(1-omega)*sum(c0[i][j] for j in CC_closest_list[i][1:n+1])/n)
            #seleziona attractiveness massima
            temp_max_index=max(enumerate(A[:]), key=lambda x:x[1])[0]  #trova indice e (valore massimo di A)      
            P_hat.append(temp_max_index) 
    
            #Trova gli SP vicini al CC con attractiveness massima
            S_accessible=[]
            for s in S_ch_set: 
                if temp_max_index in P_s_list[s]:
                    S_accessible.append(s)
            #calcola la capacità residua dei bin per assegnare gli SP più vicini al CC:
            C_res=B*C[temp_max_index]
            while len(S_accessible)!=0:
                #trova indice di SP più vicino (ERCA UN METODO MIGLIORE?)     
                s_star=list(set(np.where(cs==min(cs[list(S_accessible) ,temp_max_index]))[0]).intersection(S_accessible))[0]  
                S_accessible=set(S_accessible)-set([s_star])
                if C_res>=d_s[s_star]: #controlla se SP produce meno olio della capacità residua
                    C_res=C_res-d_s[s_star]
                    S_ch_set=set(S_ch_set)-set([s_star]) #esclude SP appena assegnati
        
        #costruisco soluzione con AP-S o AP-F
        
        #P_hat_s è il set dei CC entro la distanza di copertura dal SP s
        #quindi per ogni SP creo lista P_s contente indice dei CC compatibili, poi salvo in lista di liste il tutto
        P_hat_s_list=[]  #inizializzo lista che conterrà i vari P_hat_s
        for s in range(len(S)):
            P_hat_s=[]  #inizializzo P_hat_s
            for i in (P_hat): 
                if (distance(S[s], P[i])<300):   #controlla se la distanza è minore di 300
                    P_hat_s.append(i)  #se distanza e minore di 300 salva l'indice del CC compatibile
            P_hat_s_list.append(P_hat_s)  #aggiunge la lista alla lista totale
            
            
        #define the AP-S (and AP-F)
        #crea i set degli indici
        AP=gp.Model() #crea il modello
        AP.modelSense = gp.GRB.MINIMIZE

        #decision variables
        #1 se il CC i viene aperto, 0 altrimenti
        x_i=AP.addVars([(i) for i in P_hat], vtype=gp.GRB.BINARY) 

        #1 se il SP s viene assegnato al CC i, 0 altrimenti.
        z_si=AP.addVars([(s, i) for s in range(len(S)) for i in P_hat_s_list[s]], vtype=gp.GRB.BINARY) 

        #numero di bins in CC i
        y_i=AP.addVars([(i) for i in P_hat], lb=0, vtype=gp.GRB.INTEGER) 


        #objective function and constraints
        #20)objective function
        AP.setObjective(gp.quicksum(f[i]*x_i[i] for i in P_hat)+
                        b*gp.quicksum(y_i[i] for i in P_hat)
                        )

        #21)nel paper è sbagliato, z_si[s, i] i dipende da s e non esiste per tutti elementi P_hat
        #definisco allo stesso modo del constraint 2 del MIP
        AP.addConstrs(gp.quicksum(d_s[s]*z_si[s, i] for s in range(len(S)) if i in P_hat_s_list[s])<=
                      B*C[i]*x_i[i]
                      for i in P_hat 
                      )

        #22)
        AP.addConstrs(z_si[s,i]<=
                      x_i[i]
                      for s in S_set for i in P_hat_s_list[s] 
                      )    

        #23) 
        AP.addConstrs(gp.quicksum(z_si[s,i] for i in P_hat_s_list[s])==1 
                      for s in range(len(S))  
                      )   

        #24)nel paper è sbagliato, z_si[s, i] i dipende da s e non esiste per tutti elementi P_hat
        #definisco allo stesso modo del constraint 2 del MIP
        AP.addConstrs(gp.quicksum(d_s[s]*z_si[s, i] for s in range(len(S)) if i in P_hat_s_list[s])<=
                      B*y_i[i]
                      for i in P_hat 
                      )

        #constraint per passare a AP-F
        #AP.addConstrs(x_i[i]==1 
        #              for i in P_hat  
        #              )   

        #25-27) sono gia definiti implicitamente nella inizializzazione delle decision variables


        #update del modello per aggiunger la funzione obbiettivo e i constraints
        AP.update()
        AP.optimize()
        #accesso soluzioni
        opened_CC=AP.getAttr('x', x_i)
        connected_SP=AP.getAttr('x', z_si)
        number_bins=AP.getAttr('x', y_i)

        #usa le soluzioni appena trovate per costruire delle routes sub-ottimali 
        #per estrarre i CC aperti
        P_open=[]
        for i in P_hat:
            if (opened_CC[i]==1):
                P_open.append(P[i])
        P_open=np.concatenate([P_open]) #mette il tutto in un unico np.array

        #per estrarre il numero di bins
        bins=[]
        for i in P_hat:
            if (opened_CC[i]==1):
                bins.append(number_bins[i])
        bins=np.concatenate([bins]) #mette il tutto in un unico np.array

        #versione modificata di algoritmo Clarke-Wright savings che tiene anche conto della lunghezza massima di un percorso
        #calcola la matrice dei savings
        savings=np.empty(shape=(len(P_open),len(P_open)))
        for i in range(len(P_open)):
            for j in range(len(P_open)):
                if i==j:
                    savings[i,j]=0
                else:    
                    savings[i,j]=distance(biodiesel[0], P_open[i])+distance(biodiesel[0], P_open[j])-distance(P_open[i], P_open[j])

        #ordina in ordine decrescente i savings (considerando savings[i,j]=savings[j,i])
        links=[]
        for i in range(len(P_open)):
            for j in range(i+1, len(P_open)):
                links.append([savings[i,j], [i,j]])
        links=sorted(links, key=lambda x: x[0], reverse=True)  #ordina i savings e il link in ordine decrescente
        links=[row[1] for row in links] #ritorna solo i links del percorso

        routes = [[i] for i in range(len(P_open))]
        for i in [row[0] for row in links]:
            for j in [row[1] for row in links]:
                route_i, route_j = None, None
                #Find the routes containing CC i and j
                for route in routes: 
                    if i in route:
                        route_i = route
                    if j in route:
                        route_j = route
                    
                    #If i and j are in different routes and merging them does not violate capacity and duration constraints
                    if (route_i != None) and (route_j != None):
                        #calcola lunghezza percorso
                        totale_i=0
                        for o in range(len(P_open[route_i])-1):
                            totale_i=distance(P_open[route_i][o], P_open[route_i][o+1])
                        totale_i+=distance(P_open[route_i][0],biodiesel[0])+distance(P_open[route_i][len(route_i)-1],biodiesel[0]) #add the distance from the biodiesel facility
                    
                        totale_j=0
                        for o in range(len(P_open[route_j])-1):
                            totale_j=distance(P_open[route_j][o], P_open[route_j][o+1])
                        totale_j+=distance(P_open[route_j][0],biodiesel[0])+distance(P_open[route_j][len(route_j)-1],biodiesel[0]) #add the distance from the biodiesel facility                        
                        
                        try: #errore che non so risolvere, avviene in una sola iterazione,interromperebbe esecuzione, ma si arriva a soluzione lo stesso
                            if (route_i != route_j) and (sum(bins[route_i]) + sum(bins[route_j]) <= Q) and (totale_i+totale_j)<=1800:
                                route_i_index = routes.index(route_i)
                                route_j_index = routes.index(route_j)
                                #Merge the routes
                                routes[route_i_index] += routes[route_j_index]
                                #Remove the second route
                                routes.pop(route_j_index)
                        except: 
                            continue
        for i in range(len(routes)):
            routes[i].insert(0,len(P0)) #aggiunge il centro biodiesel a inizio route
            routes[i].append(len(P0))  #aggiunge il centro biodiesel a fine route
        #Gli indici della route corrispondon a opened_CC, in VNS bisogna convertirli a indici di P/P0    
        #aggiunge a lista i risultati trovati per i valori correnti di omega e n
        beta.append([opened_CC, connected_SP, number_bins, routes]) 
        AP.dispose()   #resetta il modello

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-26
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 568 rows, 468 columns and 2212 nonzeros
Model fingerprint: 0x1b20bcf3
Variable types: 0 continuous, 468 integer (452 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1955.0000000
Presolve removed 46 rows and 4 columns
Presolve time: 0.01s
Presolved: 522 rows, 464 columns, 2082 nonzeros
Variable types: 0 continuous, 464 integer (448 binary)

Root relaxation: objective 6.399150e+02, 517 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  639.91500    0   46 1935.00000  639.91500  66.9%     -    0s
H    0     0                     685.0000000  639.91500  6.58%     -    0s
H    0     0                     675.0000000  639.91500  5.20%     -    0s
     0     0  650.35027    0   46  675.00000  650.35027  3.65%     -    0s
     0     0  650.35027    0   47  675.00000  650.35027  3.65%     -    0s
     0     0  656.00082    0  115  675.00000  656.00082  2.81%     -    0s
     0     0  656.00082    0   48  675.00000  656.00082  2.81%     -    0s
     0     0  656.16741    0  137  675.00000  656.16741  2.79%     -    0s
     0     0  656.76257    0  140  675.00000  656.76257  2.70%     -    0s
     0     0  656.79172    0  140  675.00000  656.79172  2.70%     -    0s
     0     0  665.80000    0    6  675.00000  665.80000  1.36%     -    0s
     0     0  665.80000

     0     0  654.89798    0  157  675.00000  654.89798  2.98%     -    0s
     0     0  655.99443    0  162  675.00000  655.99443  2.82%     -    0s
     0     0  656.04580    0  162  675.00000  656.04580  2.81%     -    0s
     0     0  659.96892    0  169  675.00000  659.96892  2.23%     -    0s
     0     0  660.83341    0  170  675.00000  660.83341  2.10%     -    0s
     0     0  660.85423    0  171  675.00000  660.85423  2.10%     -    0s
     0     0  660.90994    0  170  675.00000  660.90994  2.09%     -    0s
     0     0  660.90994    0  170  675.00000  660.90994  2.09%     -    0s
     0     0  665.80000    0    9  675.00000  665.80000  1.36%     -    0s
     0     0  665.80000    0   10  675.00000  665.80000  1.36%     -    0s
     0     0  665.80000    0   12  675.00000  665.80000  1.36%     -    0s
     0     0  665.80000    0   11  675.00000  665.80000  1.36%     -    0s
     0     0  665.80000    0    4  675.00000  665.80000  1.36%     -    0s
     0     0  665.80000  

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 568 rows, 468 columns and 2212 nonzeros
Model fingerprint: 0x6bc7a860
Variable types: 0 continuous, 468 integer (452 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1965.0000000
Presolve removed 46 rows and 4 columns
Presolve time: 0.01s
Presolved: 522 rows, 464 columns, 2082 nonzeros
Variable types: 0 continuous, 464 integer (448 binary)

Root relaxation: objective 6.399150e+02, 475 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  639.91

Found heuristic solution: objective 1945.0000000
Presolve removed 46 rows and 4 columns
Presolve time: 0.01s
Presolved: 522 rows, 464 columns, 2082 nonzeros
Variable types: 0 continuous, 464 integer (448 binary)

Root relaxation: objective 6.399150e+02, 443 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  639.91500    0   44 1945.00000  639.91500  67.1%     -    0s
H    0     0                     804.0000000  639.91500  20.4%     -    0s
H    0     0                     789.0000000  639.91500  18.9%     -    0s
H    0     0                     685.0000000  639.91500  6.58%     -    0s
H    0     0                     675.0000000  639.91500  5.20%     -    0s
     0     0  653.86926    0  182  675.00000  653.86926  3.13%     -    0s
     0     0  655.14121    0  156  675.00000  655.14121  2.94%     -    0s
     0     0  657.3622

...

     0     0  700.13242    0  173  728.00000  700.13242  3.83%     -    0s
     0     0  704.67100    0  139  728.00000  704.67100  3.20%     -    0s
     0     0  711.70267    0  152  728.00000  711.70267  2.24%     -    0s
     0     0  711.71694    0  151  728.00000  711.71694  2.24%     -    0s
     0     0  712.09348    0  156  728.00000  712.09348  2.18%     -    0s
     0     0  712.23709    0  190  728.00000  712.23709  2.17%     -    0s
     0     0  712.28100    0  194  728.00000  712.28100  2.16%     -    0s
     0     0  712.38935    0  183  728.00000  712.38935  2.14%     -    0s
     0     0  712.42245    0  182  728.00000  712.42245  2.14%     -    0s
     0     0  712.47136    0  183  728.00000  712.47136  2.13%     -    0s
     0     0  712.50613    0  182  728.00000  712.50613  2.13%     -    0s
H    0     0                     718.0000000  712.50613  0.77%     -    0s
     0     0  713.89504    0   95  718.00000  713.89504  0.57%     -    0s
     0     0  713.89504  

Thread count was 4 (of 4 available processors)

Solution count 4: 718 728 738 1656 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.180000000000e+02, best bound 7.180000000000e+02, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 551 rows, 451 columns and 2143 nonzeros
Model fingerprint: 0x11404734
Variable types: 0 continuous, 451 integer (437 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1656.0000000
Presolve removed 66 rows and 14 columns
Presolve time: 0.01s
Presolved: 485 rows, 437 columns, 1992 nonzeros
Variable types: 0 continuous, 437 integer (423 binary)

Root relaxation: objective 6.891800e+02, 405 iterati

  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1656.0000000
Presolve removed 66 rows and 14 columns
Presolve time: 0.01s
Presolved: 485 rows, 437 columns, 1992 nonzeros
Variable types: 0 continuous, 437 integer (423 binary)

Root relaxation: objective 6.891800e+02, 405 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  689.18000    0   43 1656.00000  689.18000  58.4%     -    0s
H    0     0                     738.0000000  689.18000  6.62%     -    0s
H    0     0                     728.0000000  689.18000  5.33%     -    0s
H    0     0                     718.0000000  689.18000  4.01%     -    0s
     0     0  691.86154    0   49  718.00000  691.86154  3.64%     -    0s
     0     0  691.86154    0   38  


     0     0  689.18000    0   43 1656.00000  689.18000  58.4%     -    0s
H    0     0                     738.0000000  689.18000  6.62%     -    0s
H    0     0                     728.0000000  689.18000  5.33%     -    0s
H    0     0                     718.0000000  689.18000  4.01%     -    0s
     0     0  691.86154    0   49  718.00000  691.86154  3.64%     -    0s
     0     0  691.86154    0   38  718.00000  691.86154  3.64%     -    0s
     0     0  692.93684    0   45  718.00000  692.93684  3.49%     -    0s
     0     0 infeasible    0       718.00000  718.00000  0.00%     -    0s

Cutting planes:
  Gomory: 2
  MIR: 2

Explored 1 nodes (1030 simplex iterations) in 0.10 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 718 728 738 1656 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.180000000000e+02, best bound 7.180000000000e+02, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD R


Cutting planes:
  Gomory: 2
  MIR: 2

Explored 1 nodes (1030 simplex iterations) in 0.10 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 718 728 738 1656 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.180000000000e+02, best bound 7.180000000000e+02, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen 3 3250U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 551 rows, 451 columns and 2143 nonzeros
Model fingerprint: 0x11404734
Variable types: 0 continuous, 451 integer (437 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1656.0000000
Presolve removed 66 rows and 14 columns
Presolve time: 0.01s
Presolved: 485 rows, 437 columns, 1992 n

## VNS

Implementazione di una variable neighborhood search utilizzando le soluzioni appena trovate con constructive heuristic.

La VNS utilizza quattro diverse operazioni:
-)inserisce un segmento di una route in un altra route (insert)
-)scambia segementi di due route(swap)
-)rimuove un segmento di CCs da una route e aggiunge un numero di CCs non aperti alla route (insert & remove)
-)rimuove un CC dalla route (Remove)